# Bayesian analyses and MCMC

In [3]:
"""import sys
import os

# Adicionar o caminho de 'src' para o sys.path
sys.path.append(os.path.abspath('../CMBA'))"""

"import sys\nimport os\n\n# Adicionar o caminho de 'src' para o sys.path\nsys.path.append(os.path.abspath('../CMBA'))"

## Defining priors

In [1]:
from likelihood import JointLikelihoodFunction, JointPriors
from equations import DM_EXT_model, Hubble, Modulus_sne

model = DM_EXT_model()
wCDM = Hubble()
mu_wCDM = Modulus_sne()

# Definindo os priors conjuntos
param_configs_constant = {
    'H_0': ((55, 91), 'uniform'),
    'Omega_m': ((0.26, 0.36), 'gaussian'),
    'A': ((55, 255), 'gaussian'),
    'beta': ((-5, 5), 'gaussian'),
    'omega_0': ((-2, 0), 'uniform')
}

param_configs_w0wa = {
    'H_0': ((55, 91), 'uniform'),
    'Omega_m': ((0.26, 0.36), 'gaussian'),
    'A': ((55, 255), 'gaussian'),
    'beta': ((-5, 5), 'gaussian'),
    'omega_0': ((-2, 0), 'uniform'),
    'omega_a': ((-2, 2), 'uniform')
}

P_constant = JointPriors(param_configs_constant)

P_w0wa = JointPriors(param_configs_w0wa)

# Creating an instance of JointLikelihoodFunction
LF_constant = JointLikelihoodFunction(
    { 'FRB': lambda z, H_0, Omega_m, A, beta, omega_0: model.DM_ext_th(
        z=z,
        f_IGM=0.83,
        model_type='constant',
        Omega_b=None,  
        Omega_m=Omega_m,     
        H_today=H_0,
        A=A,
        beta=beta,
        omega_0=omega_0,  
        cosmo_type='non_standard',
        param_type='constant'
    ), 

    'H(z)': lambda z, H_0, Omega_m, omega_0: wCDM.H_func(
        z=z,
        Omega_m=Omega_m,
        H_0=H_0,
        omega_0=omega_0,
        cosmo_type='non_standard',
        param_type='constant'
    ),

    'SNe': lambda z, H_0, Omega_m, omega_0: mu_wCDM.Modulo_std(
        z=z,
        Omega_m=Omega_m,
        H_0=H_0,
        omega_0=omega_0,
        cosmo_type='non_standard',
        param_type='constant'
    ) 
    }
)

LF_CPL = JointLikelihoodFunction(
    { 'FRB': lambda z, H_0, Omega_m, A, beta, omega_0, omega_a: model.DM_ext_th(
        z=z,
        f_IGM=0.83,
        model_type='constant',
        Omega_b=None,  
        Omega_m=Omega_m,     
        H_today=H_0,
        A=A,
        beta=beta,
        omega_0=omega_0,
        omega_a=omega_a,  
        cosmo_type='non_standard',
        param_type='CPL'
    ), 

    'H(z)': lambda z, H_0, Omega_m, omega_0, omega_a: wCDM.H_func(
        z=z,
        Omega_m=Omega_m,
        H_0=H_0,
        omega_0=omega_0,
        omega_a=omega_a,
        cosmo_type='non_standard',
        param_type='CPL'
    ),

    'SNe': lambda z, H_0, Omega_m, omega_0, omega_a: mu_wCDM.Modulo_std(
        z=z,
        Omega_m=Omega_m,
        H_0=H_0,
        omega_0=omega_0,
        omega_a=omega_a,
        cosmo_type='non_standard',
        param_type='CPL'
    ) 
    }
)

LF_BA = JointLikelihoodFunction(
    { 'FRB': lambda z, H_0, Omega_m, A, beta, omega_0, omega_a: model.DM_ext_th(
        z=z,
        f_IGM=0.83,
        model_type='constant',
        Omega_b=None,  
        Omega_m=Omega_m,     
        H_today=H_0,
        A=A,
        beta=beta,
        omega_0=omega_0,
        omega_a=omega_a,  
        cosmo_type='non_standard',
        param_type='BA'
    ), 

    'H(z)': lambda z, H_0, Omega_m, omega_0, omega_a: wCDM.H_func(
        z=z,
        Omega_m=Omega_m,
        H_0=H_0,
        omega_0=omega_0,
        omega_a=omega_a,
        cosmo_type='non_standard',
        param_type='BA'
    ),

    'SNe': lambda z, H_0, Omega_m, omega_0, omega_a: mu_wCDM.Modulo_std(
        z=z,
        Omega_m=Omega_m,
        H_0=H_0,
        omega_0=omega_0,
        omega_a=omega_a,
        cosmo_type='non_standard',
        param_type='BA'
    ) 
    }
)

ModuleNotFoundError: No module named 'likelihood'

## Preparing the samples

### Analysis for 16 FRBs

In [ ]:
from obs_data import FRB_data, H_data, SNe_data
import ultranest

# Instantiate the FRB_data class for 16 FRBs
frb_data_16 = FRB_data(n_frb=16)

Hz_data = H_data()

sne_data = SNe_data(sample_sne='Pantheon+')

# Call the select_data method to get the observed data
z_obs_16, DM_obs_ext_16, DM_obs_ext_error_16 = frb_data_16.select_data()

z_obs_hz, H_obs, H_obs_error = Hz_data.H_z_data()

z_sne, mu_sne, cov_matrix = sne_data.load_data()

# Configuring the ultranest samplers
sampler_constant_16 = ultranest.ReactiveNestedSampler(
    P_constant.param_names,
    lambda params: LF_constant.log_likelihood(
        dict(zip(P_constant.param_names, params)),
        {
            'FRB': (z_obs_16, DM_obs_ext_16, DM_obs_ext_error_16),
            #'H(z)': (z_obs_hz, H_obs, H_obs_error),
            #'SNe': (z_sne, mu_sne, cov_matrix)
        }
    ),
    P_constant.prior_transform
)

sampler_CPL_16 = ultranest.ReactiveNestedSampler(
    P_w0wa.param_names,
    lambda params: LF_CPL.log_likelihood(
        dict(zip(P_w0wa.param_names, params)),
        {
            'FRB': (z_obs_16, DM_obs_ext_16, DM_obs_ext_error_16),
            #'H(z)': (z_obs_hz, H_obs, H_obs_error),
            #'SNe': (z_sne, mu_sne, cov_matrix)
        }
    ),
    P_w0wa.prior_transform
)

sampler_BA_16 = ultranest.ReactiveNestedSampler(
    P_w0wa.param_names,
    lambda params: LF_BA.log_likelihood(
        dict(zip(P_w0wa.param_names, params)),
        {
            'FRB': (z_obs_16, DM_obs_ext_16, DM_obs_ext_error_16),
            #'H(z)': (z_obs_hz, H_obs, H_obs_error),
            #'SNe': (z_sne, mu_sne, cov_matrix)
        }
    ),
    P_w0wa.prior_transform
)

### Analysis for 66 FRBs

In [4]:
# Instantiate the FRB_data class for 66 FRBs
frb_data_63 = FRB_data(n_frb=66)

# Call the select_data method to get the observed data
z_obs_63, DM_obs_ext_63, DM_obs_ext_error_63 = frb_data_63.select_data()

# Configuring the ultranest samplers
sampler_constant_63 = ultranest.ReactiveNestedSampler(
    P_constant.param_names,
    lambda params: LF_constant.log_likelihood(
        dict(zip(P_constant.param_names, params)),
        {
            'FRB': (z_obs_63, DM_obs_ext_63, DM_obs_ext_error_63),
            #'H(z)': (z_obs_hz, H_obs, H_obs_error),
            #'SNe': (z_sne, mu_sne, cov_matrix)
        }
    ),
    P_constant.prior_transform
)

sampler_CPL_63 = ultranest.ReactiveNestedSampler(
    P_w0wa.param_names,
    lambda params: LF_CPL.log_likelihood(
        dict(zip(P_w0wa.param_names, params)),
        {
            'FRB': (z_obs_63, DM_obs_ext_63, DM_obs_ext_error_63),
            #'H(z)': (z_obs_hz, H_obs, H_obs_error),
            #'SNe': (z_sne, mu_sne, cov_matrix)
        }
    ),
    P_w0wa.prior_transform
)

sampler_BA_63 = ultranest.ReactiveNestedSampler(
    P_w0wa.param_names,
    lambda params: LF_BA.log_likelihood(
        dict(zip(P_w0wa.param_names, params)),
        {
            'FRB': (z_obs_63, DM_obs_ext_63, DM_obs_ext_error_63),
            #'H(z)': (z_obs_hz, H_obs, H_obs_error),
            #'SNe': (z_sne, mu_sne, cov_matrix)
        }
    ),
    P_w0wa.prior_transform
)

In [ ]:
wCDM_16 = sampler_constant_16.run(min_num_live_points=400)
sampler_constant_16.print_results()

In [ ]:
wCDM_63 = sampler_constant_63.run(min_num_live_points=400)
sampler_constant_63.print_results()

In [ ]:
CPL_16 = sampler_CPL_16.run(min_num_live_points=400)
sampler_CPL_16.print_results()

In [ ]:
CPL_63 = sampler_CPL_63.run(min_num_live_points=400)
sampler_CPL_63.print_results()

In [ ]:
BA_16 = sampler_BA_16.run(min_num_live_points=400)
sampler_BA_16.print_results()

In [ ]:
BA_63 = sampler_BA_63.run(min_num_live_points=400)
sampler_BA_63.print_results()

In [ ]:
from getdist import plots, MCSamples

# Extraindo amostras dos resultados
samples1_16 = wCDM_16['samples']
samples2_16 = CPL_16['samples']
samples3_16 = BA_16['samples']

samples1_63 = wCDM_63['samples']
samples2_63 = CPL_63['samples']
samples3_63 = BA_63['samples']

# Criando objetos MCSamples com os dados
labels1 = ['H_0', '\\Omega_m', 'A', '\\beta', '\\omega_0']
names1 = P_constant.param_names
mcsamples1_16 = MCSamples(samples=samples1_16, names=names1, labels=labels1)
mcsamples1_63 = MCSamples(samples=samples1_63, names=names1, labels=labels1)

labels2 = ['H_0', '\\Omega_m', 'A', '\\beta', '\\omega_0', '\\omega_a']
names2 = P_w0wa.param_names
mcsamples2_16 = MCSamples(samples=samples2_16, names=names2, labels=labels2)
mcsamples2_63 = MCSamples(samples=samples2_63, names=names2, labels=labels2)

mcsamples3_16 = MCSamples(samples=samples3_16, names=names2, labels=labels2)
mcsamples3_63 = MCSamples(samples=samples3_63, names=names2, labels=labels2)

In [ ]:
# Plotando os Triangle plots
g = plots.get_subplot_plotter()
mcsamples1_16.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples1_63.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
g.settings.num_plot_contours = 2
g.triangle_plot([mcsamples1_16, mcsamples1_63], filled=True, contour_colors=['green', 'red'], 
                legend_labels=['16 FRBs', '66 FRBs'])
g.export('Figuras/param_constant_frb.png', dpi=600)

In [ ]:
# Filled 2D comparison plot with legend
g = plots.get_single_plotter(width_inch=4, ratio=1)
g.plot_2d([mcsamples1_16, mcsamples1_63], 'H_0', 'omega_0', filled=True, colors=['green', 'red'])
g.add_legend(['16 FRBs', '66 FRBs'], legend_loc='lower right')
g.export('Figuras/param_constant_frb_w0_H0.png', dpi=600)

In [ ]:
# Plotando os Triangle plots
g = plots.get_subplot_plotter()
mcsamples2_16.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples2_63.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
g.settings.num_plot_contours = 2
g.triangle_plot([mcsamples2_16, mcsamples2_63], filled=True, contour_colors=['green', 'red'], 
                legend_labels=['CPL with 16 FRBs', 'CPL with 66 FRBs'],) 
                #title_limit=1)
g.export('Figuras/param_CPL_frb.png', dpi=600)

In [ ]:
# Filled 2D comparison plot with legend
g = plots.get_single_plotter(width_inch=4, ratio=1)
g.plot_2d([mcsamples2_16, mcsamples2_63], 'H_0', 'omega_0', filled=True, colors=['green', 'red'])
g.add_legend(['16 FRBs', '66 FRBs'], legend_loc='lower right')
g.export('Figuras/param_CPL_frb_w0_H0.png', dpi=600)

In [ ]:
# Plotando os Triangle plots
g = plots.get_subplot_plotter()
mcsamples3_16.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples3_63.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
g.settings.num_plot_contours = 2
g.triangle_plot([mcsamples3_16, mcsamples3_63], filled=True, contour_colors=['green', 'red'], 
                legend_labels=['BA with 16 FRBs', 'BA with 66 FRBs'],)
                #title_limit=1)
g.export('Figuras/param_BA_frb.png', dpi=600)

In [ ]:
# Filled 2D comparison plot with legend
g = plots.get_single_plotter(width_inch=4, ratio=1)
g.plot_2d([mcsamples3_16, mcsamples3_63], 'H_0', 'omega_0', filled=True, colors=['green', 'red'])
g.add_legend(['16 FRBs', '66 FRBs'], legend_loc='lower right')
g.export('Figuras/param_BA_frb_w0_H0.png', dpi=600)

## Model Comparison

In [18]:
samples = {
    "Sample_1": {
        "results": {
            'wCDM': {'logz': wCDM_16['logz'], 'num_params': 5},
            'CPL': {'logz': CPL_16['logz'], 'num_params': 6},
            'BA': {'logz': BA_16['logz'], 'num_params': 6}
        },
        "n_data": 16
    },
    "Sample_2": {
        "results": {
            'wCDM': {'logz': wCDM_63['logz'], 'num_params': 5},
            'CPL': {'logz': CPL_63['logz'], 'num_params': 6},
            'BA': {'logz': BA_63['logz'], 'num_params': 6}
        },
        "n_data": 66
    }
}


In [19]:
from bayesian_analysis import ModelComparison, SaveResults

comparison = ModelComparison(samples)
comparison.run_comparisons(save_to_file='comparisons_output.txt')

In [ ]:
# Initialize the class with the desired output file
results_saver = SaveResults("model_results.txt")

# Optional: Clear the file if you want to start fresh
results_saver.reset_file()

# Save results for models with 16 live points
results_saver.save_to_txt(mcsamples1_16, 'wCDM_16')
results_saver.save_to_txt(mcsamples2_16, 'CPL_16')
results_saver.save_to_txt(mcsamples3_16, 'BA_16')

# Save results for models with 63 live points
results_saver.save_to_txt(mcsamples1_63, 'wCDM_63')
results_saver.save_to_txt(mcsamples2_63, 'CPL_63')
results_saver.save_to_txt(mcsamples3_63, 'BA_63')


In [ ]:
from ultranest.plot import PredictionBand
from equations import DM_EXT_model
import matplotlib.pyplot as plt
import numpy as np

plt.figure()
plt.xlabel('$z$')
plt.ylabel('$DM_{ext}(z)$')
plt.errorbar(x=z_values_16, y=dm_ext_obs_16, fmt='o', alpha=0.6, color='red', label='16 FRBs', ms=2)

z_test = np.linspace(0, 1, 100)

band = PredictionBand(z_test)
model_fit = DM_EXT_model()
# go through the solutions
for H_0, A, beta, omega_0, omega_a  in sampler_p2_16.results['samples']:
    # compute for each time the y value
    band.add(model_fit.DM_ext_th(z=z_test,
        f_IGM=0.83,
        model_type='constant',
        Omega_b=None,  
        Omega_m=None,     
        H_today=H_0,
        A=A,
        beta=beta,
        omega_0=omega_0,  
        omega_a=omega_a,
        cosmo_type='non_standard',
        param_type='CPL'))

band.line(color='k', linestyle='-', label='CPL parameterization', linewidth=1.5)
# add 1 sigma quantile
band.shade(color='green', alpha=0.3)
# add wider quantile (0.01 .. 0.99)
band.shade(q=0.49, color='green', alpha=0.2)

plt.legend()
plt.savefig('Figuras/DM_ext_bestfit_16.png', format='png', dpi=600)

In [ ]:
from ultranest.plot import PredictionBand
from equations import DM_EXT_model
import matplotlib.pyplot as plt
import numpy as np

plt.figure()
plt.xlabel('$z$')
plt.ylabel('$DM_{ext}(z)$')
plt.errorbar(x=z_values_63, y=dm_ext_obs_63, fmt='o', alpha=0.6, color='red', label='63 FRBs', ms=2)

z_test = np.linspace(0, 1.1, 100)

band = PredictionBand(z_test)
model_fit = DM_EXT_model()
# go through the solutions
for H_0, A, beta, omega_0  in sampler_constant_63.results['samples']:
    # compute for each time the y value
    band.add(model_fit.DM_ext_th(z=z_test,
        f_IGM=0.83,
        model_type='constant',
        Omega_b=None,  
        Omega_m=None,     
        H_today=H_0,
        A=A,
        beta=beta,
        omega_0=omega_0,
        cosmo_type='non_standard',
        param_type='constant')
    )

band.line(color='k', linestyle='-', label='Constant parameterization', linewidth=1.5)
# add 1 sigma quantile
band.shade(color='green', alpha=0.3)
# add wider quantile (0.01 .. 0.99)
band.shade(q=0.49, color='green', alpha=0.2)

plt.legend()
plt.savefig('Figuras/DM_ext_bestfit_63.png', format='png', dpi=600)